<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

Основано на: https://github.com/DanAnastasyev/DeepNLP-Course Week 13


# General Conversation


In [ ]:
!pip install --upgrade pyonmttok gensim torch

     |████████████████████████████████| 2.5MB 21.4MB/s 
     |████████████████████████████████| 24.2MB 1.4MB/s 
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.7.0+cu101)
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


## Данные для русского

* Toloka Persona Chat Rus: https://toloka.yandex.ru/datasets/
* Диалоги из литературы: https://github.com/Koziev/NLP_Datasets/blob/master/Conversations/Data/dialogues.zip
* Open Subtitles:http://opus.nlpl.eu/OpenSubtitles-v2018.php

Будем работать с датасетом Толоки

In [ ]:
!rm -f TlkPersonaChatRus.zip
!rm -rf TlkPersonaChatRus
!wget https://tlk.s3.yandex.net/dataset/TlkPersonaChatRus.zip
!unzip TlkPersonaChatRus.zip
!head -n 5 TlkPersonaChatRus/dialogues.tsv

--2020-11-22 18:03:23--  https://tlk.s3.yandex.net/dataset/TlkPersonaChatRus.zip
Resolving tlk.s3.yandex.net (tlk.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to tlk.s3.yandex.net (tlk.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6139432 (5.9M) [application/zip]
Saving to: ‘TlkPersonaChatRus.zip’

TlkPersonaChatRus.z 100%[===================>]   5.85M  5.48MB/s    in 1.1s    

2020-11-22 18:03:25 (5.48 MB/s) - ‘TlkPersonaChatRus.zip’ saved [6139432/6139432]

Archive:  TlkPersonaChatRus.zip
   creating: TlkPersonaChatRus/
  inflating: TlkPersonaChatRus/dialogues.tsv  
  inflating: TlkPersonaChatRus/readme_TlkPersonaChatRus.txt  
  inflating: TlkPersonaChatRus/profiles.tsv  
persona_1_profile	persona_2_profile	dialogue
"<span class=participant_1>У меня любимая работа.<br />Я уважаю людей.<br />У меня есть животное.<br />У меня хороший друг.<br />Я люблю кофе.<br /></span>"	"<span class=participant_2>Ищу принца.<br

Парсим HTML, схлопываем подряд идущие реплики одного человека, убираем пометки о номере пользователя

In [ ]:
import csv
import copy
from html.parser import HTMLParser

class DialogueParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.lines = []
        self.current_line = ""
        self.is_line = False

    def handle_starttag(self, tag, attrs):
        if tag == "span":
            self.is_line = True

    def handle_endtag(self, tag):
        if tag == "span":
            self.lines.append(self.current_line)
            self.is_line = False
            self.current_line = ""

    def handle_data(self, data):
        if self.is_line:
            self.current_line += data
    
    def pop_dialogue(self):
        dialogue = copy.copy(self.lines)
        self.lines = []
        return dialogue


dialogues = []
parser = DialogueParser()
with open("TlkPersonaChatRus/dialogues.tsv", "r") as r:
    next(r)
    reader = csv.reader(r, delimiter='\t')
    for row in reader:
        dialogue = row[2]
        dialogue = dialogue.replace("<br />", " ").replace("<br/>", " ")
        parser.feed(dialogue)
        dialogue = parser.pop_dialogue()
        if not dialogue:
            continue
        user1_start = "Пользователь 1: "
        user2_start = "Пользователь 2: "
        for line in dialogue:
            assert line.startswith(user1_start) or line.startswith(user2_start)

        def get_user(line):
            return 1 if line.startswith(user1_start) else 2
        def clean_line(line):
            return line.replace(user1_start, "").replace(user2_start, "").replace("\n", " ").strip()

        new_dialogue = []
        current_user = get_user(dialogue[0])
        current_line = clean_line(dialogue[0])
        for line in dialogue[1:]:
            user = get_user(line)
            line = clean_line(line)
            if current_user == user:
                current_line += " " + line
                continue
            new_dialogue.append(current_line)
            current_line = line
            current_user = user
        new_dialogue.append(current_line)
        dialogues.append(new_dialogue)

In [ ]:
dialogues[0]

['Привет) расскажи о себе',
 'Привет) под вкусный кофеек настроение поболтать появилось )',
 'Что читаешь? Мне нравится классика Я тоже люблю пообщаться',
 'Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю',
 'А я выращиваю фиалки И веду здоровый и активный образ жизни!',
 'Ух ты, интересно.',
 'Ты случайно не принц на белом коне? Я его очень жду ..',
 'А у меня из хобби каждую неделю тусить с моим лучшим другом)']

Делаем примеры, состоящие из контекста и ответа

In [ ]:
blocks = []
max_context_length = 1
max_answer_length = 100
for dialogue in dialogues:
    for i in range(1, len(dialogue)):
        context = tuple(dialogue[max(0, i-max_context_length):i])
        answer = dialogue[i]
        blocks.append({"context": context, "answer": answer})
blocks = [block for block in blocks if len(block["answer"]) < max_answer_length]

In [ ]:
len(blocks)

138197

Разбиваем на обучающую и тестовую выборки

In [ ]:
import random
random.shuffle(blocks)
test_part = 0.9
border = int(len(blocks) * test_part)
train_blocks = blocks[:border]
test_blocks = blocks[border:]

BatchIterator разбивает примеры на батчи

In [ ]:
import random
import math
import torch
import numpy as np


class BatchIterator():
    def __init__(self, blocks, batch_size, vector_model, tokenizer, shuffle=True):
        self.blocks = blocks
        self.num_samples = len(blocks)
        self.batch_size = batch_size
        self.vector_model = vector_model
        self.tokenizer = tokenizer
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(len(blocks) / batch_size))
        
    def __len__(self):
        return self.batches_count
    
    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]
            pivots = []
            positives = []
            for data_ind in batch_indices:
                block = self.blocks[data_ind]
                
                pivots.append(block["context"])
                positives.append(block["answer"])

            yield {
                'pivot_lines': pivots,
                'positive_lines': positives
            }

А дальше у нас есть 2 базовых варианта: ранжирующая и порождающая модели

## Ранжирующая модель

Начнём с ранжирующей модели, она во многом проще в реализации. Нужно заранее составить большую базу ответов и просто выбирать наиболее подходящий к контексту каждый раз. 

![](https://habrastorage.org/web/c79/942/608/c79942608160404ab398033e97283c51.jpg)

*From [Neural conversational models: как научить нейронную сеть светской беседе. Лекция в Яндексе](https://habr.com/ru/company/yandex/blog/333912/)*

Сеть состоит из пары башен: левая кодирует контекст, правая - ответ. Задача - научиться считать близость между представлениями контекста и ответа.



Будем брать готовые предобученные векторы с https://rusvectores.org/

In [ ]:
!mkdir ru_fasttext
!wget http://vectors.nlpl.eu/repository/20/187.zip
!mv 187.zip ru_fasttext/187.zip
!cd ru_fasttext && unzip 187.zip
!rm ru_fasttext/187.zip

--2020-11-22 18:05:45--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  25.2MB/s    in 1m 44s  

2020-11-22 18:07:30 (24.7 MB/s) - ‘187.zip’ saved [2692389554/2692389554]

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [ ]:
import gensim
import pyonmttok

vector_model = gensim.models.KeyedVectors.load("ru_fasttext/model.model")
tokenizer = pyonmttok.Tokenizer("conservative", joiner_annotate=False)

In [ ]:
train_iter = BatchIterator(train_blocks, 64, vector_model, tokenizer)
test_iter = BatchIterator(test_blocks, 64, vector_model, tokenizer)

In [ ]:
for batch in train_iter:
    break

In [ ]:
batch['pivot_lines'][:3], batch['positive_lines'][:3]

([('Не люблю вообще. А вредную еду кушаешь?',),
  ('Здраствуйте',),
  ('У меня 4 собаки,многовато ,но я их люблю',)],
 ['Конечно, обожаю фастфуд А ты?',
  'Привет! Я с краснодарского края! Хочу свою пасеку',
  'Вот вы любите рыбалку ? Я например очень люблю рыбалку'])

Для бейзлайна архитектура башен максимально простая: cat(avg, min, max) по всем пословным векторам и умножение на матрицу. Матрицы разные для башен контекста и ответа.

In [ ]:
import torch
import torch.nn as nn

class WordVectorEncoder(nn.Module):
    def __init__(self, vector_model, tokenizer):
        super().__init__()
        self.vector_model = vector_model
        self.tokenizer = tokenizer

    def tokenize(self, text):
        text = str(text).strip().replace("\n", " ").replace("\xa0", " ").lower() # \xa0 - это неразрывный пробел.
        tokens, _ = self.tokenizer.tokenize(text)
        return tokens
    
    def to_matrix(self, lines):
        batch_size = len(lines)
        tokens = [self.tokenize(line) for line in lines]
        max_sen_len = max([len(line) for line in tokens])
        vector_size = self.vector_model.vector_size

        matrix = np.zeros((batch_size, max_sen_len, vector_size))
        for i, line in enumerate(tokens):
            for j, token in enumerate(line):
                matrix[i, j, :] = self.vector_model.get_vector(token)

        return torch.cuda.FloatTensor(matrix)
        #return torch.FloatTensor(matrix)

class NaiveEncoder(WordVectorEncoder):
    def __init__(self, vector_model, tokenizer, embedding_dim=300, output_dim=300):
        super().__init__(vector_model, tokenizer)
        self.mapping_layer = nn.Linear(embedding_dim * 3, output_dim).cuda()

    def forward(self, lines):
        in_vectors = self.to_matrix(lines)
        norm = in_vectors.norm(p=2, dim=1, keepdim=True)
        in_vectors = in_vectors.div(norm)

        avg_vectors = torch.mean(in_vectors, dim=1)
        max_vectors = torch.max(in_vectors, dim=1)[0]
        min_vectors = torch.min(in_vectors, dim=1)[0]

        projections = self.mapping_layer(torch.cat([avg_vectors, max_vectors, min_vectors], dim=1))
        norm = projections.norm(p=2, dim=1, keepdim=True)
        projections = projections.div(norm)
        return projections

В самой модели применяем 2 трюка

### Triplet Loss

Мы хотим не просто научить энкодер строить эмбеддинги для предложений. Мы хотим, чтобы притягивать векторы правильных ответов к вопросам и отталкивать неправильные. Для этого используют, например, *Triplet Loss*:

$$ L = \frac 1N \underset {q, a^+, a^-} \sum max(0, \space \delta - sim[V_q(q), V_a(a^+)] + sim[V_q(q), V_a(a^-)] ),$$

где
* $sim[a, b]$ функция похожести (например, dot product или cosine similarity)
* $\delta$ - гиперпараметр модели. Если $sim[a, b]$ линейно по $b$, то все $\delta > 0$ эквиватентны.

![img](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/margin.png)

### Hard-negatives mining

Берём в качестве отрицательного примера самый близкий из неправильных примеров в батче:
$$a^-_{hard} = \underset {a^-} {argmax} \space sim[V_q(q), V_a(a^-)]$$

Неправильные в данном случае - все, кроме правильного :)

Реализуется это как-то так:
* Батч состоит из правильных пар.
* Для всех контекстов и всех ответов считаем эмбеддинги.
* Положительные примеры у нас есть - осталось найти для каждого контекста наиболее похожие на него ответы, которые предназначались другим контекстам.

In [ ]:
class DSSMTripletLoss(nn.Module):
    def __init__(self, vector_model, tokenizer, embedding_dim=300, hidden_dim=300, margin=0.1):
        super().__init__()
        self.left_encoder = NaiveEncoder(vector_model, tokenizer, embedding_dim, hidden_dim)
        self.right_encoder = NaiveEncoder(vector_model, tokenizer, embedding_dim, hidden_dim)
        self.similarity = nn.CosineSimilarity(dim=1)
        self.margin = margin

    def apply(self, pivot_lines, positive_lines, hard_negatives_part=0.0):
        pivots = self.left_encoder(pivot_lines)
        positives = self.right_encoder(positive_lines)

        batch_size = pivots.size(0)
        shift = random.randint(1, batch_size - 1)

        negative_indices = torch.LongTensor([(i + shift) % batch_size for i in range(batch_size)])
        negatives = positives[negative_indices]

        return pivots, positives, negatives
    
    def calc_recall_at_1(self, pivots, positives, negatives):
        batch_size = pivots.size(0)

        scores = pivots.matmul(positives.transpose(0, 1))
        predicted_indices = torch.argmax(scores, dim=1)

        true_indices = torch.linspace(0, batch_size-1, steps=batch_size)
        correct_count = torch.sum(predicted_indices.cpu() == true_indices).item()
        return correct_count

    def forward(self, pivots, positives, negatives):

        distance = -self.similarity(pivots, positives) + self.similarity(pivots, negatives) + self.margin
        loss = torch.mean(torch.max(distance, torch.zeros_like(distance)))

        return loss
    
    def left_apply(self, lines):
        return self.left_encoder(lines)
  
    def right_apply(self, lines):
        return self.right_encoder(lines)

In [ ]:
class ModelTrainer():
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.correct_count = 0
        self.total_count = 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}, Recall@1 = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        pivot_lines = batch['pivot_lines']
        positive_lines = batch['positive_lines']
        pivot_lines = [" ".join(context) for context in pivot_lines]
        #loss
        pivots, positives, negatives = self.model.apply(pivot_lines, positive_lines)
        loss = self.model(pivots, positives, negatives)
        #predicts
        self.correct_count += self.model.calc_recall_at_1(pivots, positives, negatives)
        self.total_count += len(pivot_lines)
        self.epoch_loss += loss.item()
        
        if self.is_train:
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.)
            self.optimizer.step()

        return '{:>5s} Loss = {:.5f}, Recall@1 = {:.2%}'.format(
            self.name, loss.item(), self.correct_count / self.total_count
        )

In [ ]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=len(data_iter)) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if val_iter is not None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
DEVICE = torch.device('cuda')

In [ ]:
import torch.optim as optim
model = DSSMTripletLoss(vector_model, tokenizer)
optimizer = optim.Adam(model.parameters(), lr=0.003)
#collect all
trainer = ModelTrainer(model, optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=test_iter)

[1 / 30]   Val: Loss = 0.06259, Recall@1 = 9.25%: 100%|██████████| 216/216 [00:05<00:00, 42.11it/s]
[2 / 30] Train: Loss = 0.06100, Recall@1 = 10.20%: 100%|██████████| 1944/1944 [00:53<00:00, 36.32it/s]
[2 / 30]   Val: Loss = 0.05906, Recall@1 = 11.19%: 100%|██████████| 216/216 [00:05<00:00, 41.78it/s]
[3 / 30] Train: Loss = 0.05846, Recall@1 = 11.17%: 100%|██████████| 1944/1944 [00:53<00:00, 36.23it/s]
[3 / 30]   Val: Loss = 0.06006, Recall@1 = 10.93%: 100%|██████████| 216/216 [00:05<00:00, 41.43it/s]
[4 / 30] Train: Loss = 0.05638, Recall@1 = 11.82%: 100%|██████████| 1944/1944 [00:53<00:00, 36.36it/s]
[4 / 30]   Val: Loss = 0.05806, Recall@1 = 11.73%: 100%|██████████| 216/216 [00:05<00:00, 41.54it/s]
[5 / 30] Train: Loss = 0.05513, Recall@1 = 12.33%: 100%|██████████| 1944/1944 [00:53<00:00, 36.38it/s]
[5 / 30]   Val: Loss = 0.05627, Recall@1 = 12.46%: 100%|██████████| 216/216 [00:05<00:00, 41.46it/s]
[6 / 30] Train: Loss = 0.05393, Recall@1 = 12.73%: 100%|██████████| 1944/1944 [00:54

KeyboardInterrupt: ignored

In [ ]:
answers = [block["answer"] for block in blocks]
answers = list(set(answers))
random.shuffle(answers)
answers = answers[:50000]
answers_embeddings = model.right_apply(answers)

In [ ]:
answers_embeddings.shape

torch.Size([50000, 300])

In [ ]:
similarity = nn.CosineSimilarity(dim=1)
conversation = []
while True:
    user_text = input()
    conversation.append(user_text)
    conversation = conversation[-max_context_length:]

    left_embedding = model.left_apply([" ".join(conversation)])
    left_embeddings = left_embedding.expand(answers_embeddings.size(0), answers_embeddings.size(1))

    scores = similarity(left_embeddings, answers_embeddings)
    answer = answers[torch.argmax(scores, dim=0)]
    print(answer)
    conversation.append(answer)

Привет
Привет привет)
Как дела?
Как отлично все клубнично А ваши как дела?
У меня отлично тоже
Да, у меня все хорошо Работаю учителем А ваша профессия какая?
Повар
Я повар
Пока
😉 Пока пока
Как зовут?
Привет! Все отлично, как твои дела? Меня зовут Марина , а твоё имя как?


KeyboardInterrupt: ignored

## Seq2Seq
А в этом варианте просто берём модельку от машинного перевода и адаптируем её под предсказание ответа

<img src="https://www.pvsm.ru/images/2017/07/22/Neural-conversational-models-kak-nauchit-neironnuyu-set-svetskoi-besede-lekciya-v-yandekse-10.jpg">


In [ ]:
from collections import Counter
from typing import List, Tuple
import os

class Vocabulary:
    def __init__(self):
        self.index2word = list()
        self.word2index = dict()
        self.word2count = Counter()
        self.reset()

    def get_pad(self):
        return self.word2index["<pad>"]

    def get_sos(self):
        return self.word2index["<sos>"]

    def get_eos(self):
        return self.word2index["<eos>"]

    def get_unk(self):
        return self.word2index["<unk>"]

    def add_sentence(self, sentence):
        for word in sentence.strip().split():
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = len(self.index2word)
            self.word2count[word] += 1
            self.index2word.append(word)
        else:
            self.word2count[word] += 1
    
    def has_word(self, word) -> bool:
        return word in self.word2index

    def add_file(self, filename: str):
        with open(filename, "r", encoding="utf-8") as r:
            for line in r:
                for word in line.strip().split():
                    self.add_word(word)

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        return self.get_unk()

    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        empty_size = 4
        return self.size() <= empty_size

    def shrink(self, n):
        best_words = self.word2count.most_common(n)
        self.reset()
        for word, count in best_words:
            self.add_word(word)
            self.word2count[word] = count

    def reset(self):
        self.word2count = Counter()
        self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
        self.word2index = {word: index for index, word in enumerate(self.index2word)}

    def get_indices(self, sentence: str) -> List[int]:
        return [self.get_index(word) for word in sentence.strip().split()] + [self.get_eos()]

    def pad_indices(self, indices: List[int], max_length: int):
        return indices + [self.get_pad() for _ in range(max_length - len(indices))]

In [ ]:
vocabulary = Vocabulary()
for dialogue in dialogues:
    for line in dialogue:
        text = line.strip().replace("\n", " ").replace("\xa0", " ").lower()
        tokens, _ = tokenizer.tokenize(text)
        for token in tokens:
            vocabulary.add_word(token)
vocabulary.shrink(20000)

<img src="https://www.pvsm.ru/images/2017/07/22/Neural-conversational-models-kak-nauchit-neironnuyu-set-svetskoi-besede-lekciya-v-yandekse-12.jpg">

$$\operatorname{PMI}(x, y) \equiv \log \frac{p(x, y)}{p(x) p(y)}$$
---------
$$ - \operatorname{Loss}(x, y, a) = \log {p(y \mid \text x)}-a \log {p(y)} $$
---------
$$ - \operatorname{Loss}(x, y, 1) = \log {p(y \mid \text x)} - \log {p(y)} = \log \frac{p(x, y)}{p(x) p(y)} =  \operatorname{PMI}(x, y)$$
---------



<img src="https://www.pvsm.ru/images/2017/07/22/Neural-conversational-models-kak-nauchit-neironnuyu-set-svetskoi-besede-lekciya-v-yandekse-13.jpg">


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(EncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional)

    def forward(self, input_seqs, hidden=None):
        embedded = self.embedding(input_seqs)
        outputs, hidden = self.rnn(embedded, hidden)

        if self.bidirectional:
            n = hidden[0].size(0)
            hidden = (torch.cat([hidden[0][0:n:2], hidden[0][1:n:2]], 2),
                      torch.cat([hidden[1][0:n:2], hidden[1][1:n:2]], 2))
        return outputs, hidden


class Generator(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size

        self.out = nn.Linear(hidden_size, output_size)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        assert inputs.size(1) == self.hidden_size
        return self.sm(self.out(inputs))


class DecoderRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_size, output_size, max_length, n_layers=3, 
                 dropout=0.3, use_cuda=True, use_attention=True):
        super(DecoderRNN, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.use_cuda = use_cuda
        self.max_length = max_length
        self.use_attention = use_attention

        self.embedding = nn.Embedding(output_size, embedding_dim)
        
        if self.use_attention:
            self.attn = nn.Linear(embedding_dim, hidden_size)
            self.attn_sm = nn.Softmax(dim=1)
            self.attn_ctx = nn.Linear(hidden_size + hidden_size, hidden_size)
            self.attn_tanh = nn.Tanh()
        
        self.rnn = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout)
        self.generator = Generator(hidden_size, output_size)
        
    def step(self, batch_input, hidden, encoder_output):
        # batch_input: B
        # hidden: (n_layers x B x N, n_layers x B x N)
        # encoder_output: L x B x N
        # output: 1 x B x N
        # embedded:  B x E
        # attn_weights: B x 1 x L
        # context: B x 1 x N
        # rnn_input: B x N
        
        embedded = self.embedding(batch_input)
        _, hidden = self.rnn(embedded.unsqueeze(0), hidden)
        if self.use_attention:
            mapped_encoder_outputs = self.attn(encoder_output)
            h_t = hidden[0][-1].unsqueeze(2)
            mapped_encoder_outputs = mapped_encoder_outputs.transpose(0, 1)
            attn_weights = self.attn_sm(torch.bmm(mapped_encoder_outputs, h_t)).transpose(1, 2)
            max_length = encoder_output.size(0)
            context = torch.bmm(attn_weights[:, :, :max_length], encoder_output.transpose(0, 1))
            output = self.attn_tanh(self.attn_ctx(torch.cat((context, h_t.transpose(1, 2)), 2)))
        return output, hidden

    def init_state(self, batch_size, sos_index):
        initial_input = Variable(torch.zeros((batch_size,)).type(torch.LongTensor), requires_grad=False)
        initial_input = torch.add(initial_input, sos_index)
        initial_input = initial_input.cuda() if self.use_cuda else initial_input
        return initial_input

    def forward(self, current_input, hidden, length, encoder_output, gtruth=None):
        outputs = Variable(torch.zeros(length, current_input.size(0), self.output_size), requires_grad=False)
        outputs = outputs.cuda() if self.use_cuda else outputs

        for t in range(length):
            output, hidden = self.step(current_input, hidden, encoder_output)
            scores = self.generator.forward(output.squeeze(1))
            outputs[t] = scores
            if gtruth is None:
                top_indices = scores.topk(1, dim=1)[1].view(-1)
                current_input = top_indices
            else:
                current_input = gtruth[t]
        return outputs, hidden


class Seq2Seq(nn.Module):
    def __init__(self, vocabulary, embedding_dim=100,
                 rnn_size=100, encoder_n_layers=2, 
                 decoder_n_layers=2, dropout=0.3,
                 max_length=50, use_cuda=True, 
                 bidirectional=True, use_attention=True):
        super(Seq2Seq, self).__init__()

        self.vocabulary = vocabulary
        self.embedding_dim = embedding_dim
        self.output_size = vocabulary.size()
        self.rnn_size = rnn_size
        self.encoder_n_layers = encoder_n_layers
        self.decoder_n_layers = decoder_n_layers
        self.dropout = dropout
        self.max_length = max_length
        self.use_cuda = use_cuda
        self.bidirectional = bidirectional
        self.use_attention = use_attention

        self.encoder = EncoderRNN(self.output_size, embedding_dim, rnn_size, dropout=dropout,
                                  n_layers=encoder_n_layers, bidirectional=bidirectional)
        self.decoder = DecoderRNN(embedding_dim, rnn_size, self.output_size, dropout=dropout,
                                  max_length=max_length, n_layers=decoder_n_layers, use_cuda=use_cuda, 
                                  use_attention=use_attention)

    def forward(self, variable, sos_index, gtruth=None):
        encoder_output, encoder_hidden = self.encoder.forward(variable)
        current_input = self.decoder.init_state(variable.size(1), sos_index)
        max_length = self.max_length
        if gtruth is not None:
            max_length = min(self.max_length, gtruth.size(0))
        decoder_output, _ = self.decoder.forward(current_input, encoder_hidden, max_length,
                                                 encoder_output, gtruth)

        return encoder_output, decoder_output

In [ ]:
def index(vocabulary, line):
    line = line.strip().replace("\n", " ").replace("\xa0", " ").lower()
    tokens, _ = tokenizer.tokenize(line)
    return [vocabulary.get_index(token) for token in tokens]

def to_matrix(lines, vocabulary):
    indices = [index(vocabulary, line) for line in lines]
    max_len = max([len(line) for line in indices])
    matrix = np.zeros((len(indices), max_len))
    for i, line in enumerate(indices):
        matrix[i, :len(line)] = line
    return torch.cuda.LongTensor(matrix)

class Seq2SeqModelTrainer():
    def __init__(self, model, optimizer, vocabulary):
        self.model = model
        self.optimizer = optimizer
        self.criterion = nn.NLLLoss()
        self.vocabulary = vocabulary
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}'.format(self.name, self.epoch_loss / self.batches_count)
        
    def on_batch(self, batch):
        pivot_lines = batch['pivot_lines']
        positive_lines = batch['positive_lines']
        pivot_lines = [" ".join(context) for context in pivot_lines]

        input_matrix = to_matrix(pivot_lines, self.vocabulary).transpose(0, 1)
        output_matrix = to_matrix(positive_lines, self.vocabulary).transpose(0, 1)
        _, output = self.model.forward(input_matrix, self.vocabulary.get_sos(), output_matrix)
        loss = self.criterion(output.transpose(1, 2), output_matrix)
        self.epoch_loss += loss.item()
        
        if self.is_train:
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.)
            self.optimizer.step()

        return '{:>5s} Loss = {:.5f}'.format(self.name, loss.item())

In [ ]:
import torch.optim as optim

model = Seq2Seq(vocabulary).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.003)
#collect all
trainer = Seq2SeqModelTrainer(model, optimizer, vocabulary)
train_iter = BatchIterator(train_blocks, 64, vector_model, tokenizer)
test_iter = BatchIterator(test_blocks, 64, vector_model, tokenizer)
fit(trainer, train_iter, epochs_count=30, val_iter=test_iter)

[2 / 30] Train: Loss = 2.01168:  45%|████▍     | 866/1944 [02:09<02:41,  6.68it/s]


KeyboardInterrupt: ignored

In [ ]:
while True:
    user_text = input()
    input_matrix = to_matrix([user_text], vocabulary).transpose(0, 1)
    model.eval()
    _, output = model(input_matrix, model.vocabulary.get_sos())

    answer_idx = output.argmax(dim=2).cpu().numpy().ravel()
    print(' '.join(vocabulary.get_word(idx) for idx in answer_idx if idx != vocabulary.get_pad()))

Привет
привет ! как дела ?
Хорошо
отлично , а ты ?


KeyboardInterrupt: ignored

<img src="https://www.pvsm.ru/images/2017/07/22/Neural-conversational-models-kak-nauchit-neironnuyu-set-svetskoi-besede-lekciya-v-yandekse-20.jpg">

# Дополнительные материалы

## Курс Дани Анастасьева
https://github.com/DanAnastasyev/DeepNLP-Course

## Статьи
* Learning Deep Structured Semantic Models for Web Search using Clickthrough Data, 2013 [[pdf]](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/cikm2013_DSSM_fullversion.pdf)  
* Deep Learning and Continuous Representations for Natural Language Processing, Microsoft tutorial, 2016 [[pdf]](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/NAACL-HLT-2015_tutorial.pdf)
* Neural Utterance Ranking Model for Conversational Dialogue Systems, 2016 [[pdf]](https://www.aclweb.org/anthology/W16-3648.pdf)
* A Persona-Based Neural Conversation Model, 2016 [[pdf]](https://www.aclweb.org/anthology/P16-1094.pdf)

## Блоги
* [Neural conversational models: как научить нейронную сеть светской беседе](https://habr.com/company/yandex/blog/333912/)  
* [Искусственный интеллект в поиске. Как Яндекс научился применять нейронные сети, чтобы искать по смыслу, а не по словам](https://habr.com/company/yandex/blog/314222/)  
* [Triplet loss, Olivier Moindrot](https://omoindrot.github.io/triplet-loss)
* [Triplet loss in pytorch](https://discuss.pytorch.org/t/triplet-loss-in-pytorch/30634/3)